The reviews in the goodreads corpus need to be preprocessed in order to train spoilernet on it. This happens in this file

In [1]:
!pip install nltk
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
import string
import os
import pandas as pd
import tqdm

all_words = {}

def preprocess_word(w):
    if w in all_words.keys():
        return all_words[w]
    word = w.lower()
    word = word.translate(str.maketrans('', '', string.punctuation))
    word = word.replace('“', "").replace("”", "").replace("’", "").replace("…", "").replace("—", "")
    if word in stopwords.words('english'):
        return ""
    word = nltk.PorterStemmer().stem(word)
    all_words[w] = word
    return word

def preprocess_review(rev):
    new_prevs = []
    new_nexts = []
    new_sents = []
    for sent in rev.review_sentences:
        new_sent = ""
        for w in sent[1].split():
            w = preprocess_word(w)
            if len(w) > 0:
                new_sent += w + " "
        new_sents.append([sent[0],new_sent[:-1]])
    return new_sents

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_json("data/raw/goodreads_reviews_spoiler.json.gz", compression='infer', lines=True)

In [3]:
dfw = df.iloc[:200000]
for i,review in tqdm.tqdm(dfw.iterrows(),total=dfw.shape[0]):
    dfw.review_sentences[i] = preprocess_review(review)    

  0%|          | 0/200000 [00:00<?, ?it/s]/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
100%|██████████| 200000/200000 [53:56<00:00, 61.80it/s]  


In [4]:
dfw.head()

,user_id,timestamp,review_sentences,rating,has_spoiler,book_id,review_id
0,8842281e1d1347389f2ab93d60773d4d,2017-08-30,"[[0, special book], [0, start slow first third...",5,True,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb
1,8842281e1d1347389f2ab93d60773d4d,2017-03-22,"[[0, recommend katz], [0, avail free decemb ht...",3,False,16981,a5d2c3628987712d0e05c4f90798eb67
2,8842281e1d1347389f2ab93d60773d4d,2017-03-20,"[[0, fun fast pace scienc fiction thriller], [...",3,True,28684704,2ede853b14dc4583f96cf5d120af636f
3,8842281e1d1347389f2ab93d60773d4d,2016-11-09,"[[0, recommend read understand go middl americ...",0,False,27161156,ced5675e55cd9d38a524743f5c40996e
4,8842281e1d1347389f2ab93d60773d4d,2016-04-25,"[[0, realli enjoy book lot recommend], [0, dra...",4,True,25884323,332732725863131279a8e345b63ac33e


In [5]:
#dfw.to_json("pp_revs.json.gz", orient="records", lines=True, compression="infer")

In [5]:
pp_sents = []
pp_prevs = []
pp_nexts = []
spoiler = []
for i,review in tqdm.tqdm(dfw.iterrows(),total=dfw.shape[0]):
    for i in range(len(review.review_sentences)):
        pp_sents.append(review.review_sentences[i][1])
        spoiler.append(review.review_sentences[i][0])
        if i != 0:
            pp_prevs.append(review.review_sentences[i-1][1])
        else:
            pp_prevs.append("")
        if i != len(review.review_sentences)-1:
            pp_nexts.append(review.review_sentences[i+1][1])
        else:
            pp_nexts.append("")

100%|██████████| 200000/200000 [01:20<00:00, 2484.61it/s]


In [6]:
ndf = pd.DataFrame({"pp_sent":pp_sents, "spoiler":spoiler, "pp_prev":pp_prevs, "pp_next":pp_nexts})

In [7]:
ndf.to_json("data/raw/pp_context.json.gz", orient="records", lines=True, compression="infer")

In [8]:
ndf.shape

(2457349, 4)